In [100]:
#load libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import statistics

In [38]:
#import variables with stored values and store in dictionary
stored_values_df = pd.read_excel(#Insert stored variables excel here#)
stored_values_dict_df = stored_values_df[["Variable_name","Value"]]
stored_values_dict_df = stored_values_dict_df.set_index(['Variable_name'])
stored_values_dict = stored_values_dict_df.to_dict()
stored_values_dict = stored_values_dict['Value']
stored = stored_values_dict

In [40]:
#import variables with static values and store in dictionary
static_values_df = pd.read_excel(#Insert static variables excel here#)
static_values_dict_df = static_values_df[["Variable_name","Value"]]
static_values_dict_df = static_values_dict_df.set_index(['Variable_name'])
static_values_dict = static_values_dict_df.to_dict()
static_values_dict = static_values_dict['Value']
static = static_values_dict

In [22]:
#import daily inputs
daily_inputs = pd.read_excel(#Insert daily variables excel here#)

In [42]:
#create empty dataframe for outputs
column_names = ['Date','New COVID Cases','COVID Patients Entering Hospital','Total COVID in Hospital','Total PUI in Hospital','Cohorting Coefficient','COVID_patients_in_ICU', 'typical_sick_patients_in_ICU','typical_sick_ICU_currently_PUI', 'total_patients_in_ICU','COVID_patients_in_inpatient','typical_sick_in_inpatient','typical_sick_inpatient_currently_PUI','total_patients_in_inpatient','staff_working_with_COVID_PUI','staff_working_with_typical_sick','airborne_staff','total_HCWs_working_IN','gloves_used_IN','gowns_used_IN', 'N95s_used_IN','eye_pro_used_IN','surg_proc_masks_used_IN']
outputs = pd.DataFrame(columns = column_names)

In [43]:
#determine PPE use for each day in daily inputs and update outputs df

import time
start_time = time.time()

    
day = 0

for i in range(len(daily_inputs)):
    
#updates the stored value of PUI in inpatient and ICU
    
    stored['PUI_in_Inpatient'] = stored['PUI_in_Inpatient'] - (stored['PUI_in_Inpatient']/static['COVID_test_turnaround_time']) + daily_inputs.at[day,'PUI_entering_inpatient']
    stored['PUI_in_ICU'] = stored['PUI_in_ICU'] - (stored['PUI_in_ICU']/static['COVID_test_turnaround_time']) + daily_inputs.at[day,'PUI_entering_ICU']
    
    if stored['PUI_in_Inpatient'] >= static['Daily_inpatient_typical_sick']:
        stored['PUI_in_Inpatient'] = static['Daily_inpatient_typical_sick']
    if stored['PUI_in_ICU'] >= stored['Typical_sick_in_ICU']:
        stored['PUI_in_ICU'] = stored['Typical_sick_in_ICU']
    
#determines number of contact/droplet and airborne precaution patients
    Contact_Droplet_Inpatients = round((static['Daily_inpatient_typical_sick'] - stored['PUI_in_Inpatient']) * static['Percent_typical_sick_requiring_contact_droplet_precautions'])
    Contact_Droplet_ICU = round((static['Typical_sick_ICU_occupancy'] - stored['PUI_in_ICU']) * static['Percent_typical_sick_requiring_contact_droplet_precautions'])
    Contact_ED = round((static['Daily_patients_presenting_to_ED']* static['Percent_typical_sick_requiring_contact_droplet_precautions']))
    
    Airborne_Inpatients = round((static['Daily_inpatient_typical_sick'] - stored['PUI_in_Inpatient']) * static['Percent_typical_sick_requiring_airborne_precautions'])
    Airborne_ICU = round((static['Typical_sick_ICU_occupancy'] - stored['PUI_in_ICU']) * static['Percent_typical_sick_requiring_contact_droplet_precautions'])
    
    Standard_Inpatients = round(static['Daily_inpatient_typical_sick'] - stored['PUI_in_Inpatient'] - Contact_Droplet_Inpatients - Airborne_Inpatients)
    Standard_ICU = round(static['Typical_sick_ICU_occupancy'] - stored['PUI_in_ICU'] - Contact_Droplet_ICU - Airborne_ICU)
    
#calculates AGPs
    AGPs = daily_inputs.at[day,'COVID_AGPs_performed']
    AGP_HCWs = AGPs * static['HCWs_per_AGP']
    
#calculates cohorting coefficient and staff per patient for each staff and patient type
    
    Total_PUI_and_COVID = stored['PUI_in_Inpatient'] + stored['PUI_in_ICU'] + stored['COVID_in_ICU'] + stored['COVID_inpatient']
    Total_Typical_Sick = (typical_sick_inpatient - stored['PUI_in_Inpatient']) + (stored['Typical_sick_in_ICU'] - stored['PUI_in_ICU'])
    Cohorting = Total_PUI_and_COVID/ (Total_PUI_and_COVID + Total_Typical_Sick)
   
    
    RNs_per_ICU_patient = static['RNs_per_ICU_patient'] + ((1-Cohorting)*(1-static['RNs_per_ICU_patient']))
    RNs_per_Inpatient = static['RNs_per_Inpatient'] + ((1-Cohorting)*(1-static['RNs_per_Inpatient']))
    MDs_per_ICU_patient = static['MDs_per_ICU_patient'] + ((1-Cohorting)*(1-static['MDs_per_ICU_patient']))
    MDs_per_Inpatient = static['MDs_per_Inpatient'] + ((1-Cohorting)*(1-static['MDs_per_Inpatient']))
    PCA_per_ICU_patient = static['PCA_per_ICU_patient'] + ((1-Cohorting)*(1-static['PCA_per_ICU_patient']))
    PCA_per_Inpatient = static['PCA_per_Inpatient'] + ((1-Cohorting)*(1-static['PCA_per_Inpatient']))
    Student_per_ICU_patient = static['Student_per_ICU_patient'] + ((1-Cohorting)*(1-static['Student_per_ICU_patient']))
    Student_per_Inpatient = static['Student_per_Inpatient'] + ((1-Cohorting)*(1-static['Student_per_Inpatient']))
    Non_Clinical_per_patient = static['Non_Clinical_per_patient'] + ((1-Cohorting)*(1-static['Non_Clinical_per_patient']))

#calculates total staff working
    def staff_ICU_Typical_Sick_Standard(staff_per_ICU_patient):
        return round(Standard_ICU * staff_per_ICU_patient) * static['Shifts_per_day_clinical']
    
    def staff_ICU_Typical_Sick_ContactDroplet(staff_per_ICU_patient):
        return round(Contact_Droplet_ICU * staff_per_ICU_patient) * static['Shifts_per_day_clinical']
    
    def staff_ICU_Typical_Sick_Airborne():
        return round(Airborne_ICU * 1) * static['Shifts_per_day_clinical']
    
    def staff_ICU_COVID_PUI(staff_per_ICU_patient):
        return round((stored['COVID_in_ICU'] + stored['PUI_in_ICU'])* staff_per_ICU_patient)* static['Shifts_per_day_clinical']
    
    def staff_Inpatient_Typical_Sick_Standard(staff_per_Inpatient):
        return round(Standard_Inpatients * staff_per_Inpatient) * static['Shifts_per_day_clinical']
    
    def staff_Inpatient_Typical_Sick_ContactDroplet(staff_per_Inpatient):
        return round(Contact_Droplet_Inpatients * staff_per_Inpatient) * static['Shifts_per_day_clinical']
    
    def staff_Inpatient_Typical_Sick_Airborne():
        return round(Airborne_Inpatients * 1) * static['Shifts_per_day_clinical']
    
    def staff_Inpatient_COVID_PUI(staff_per_Inpatient):
        return round((stored['COVID_inpatient'] + stored['PUI_in_Inpatient'])* staff_per_Inpatient) * static['Shifts_per_day_clinical']
    
    
    HCWname = ['RN', 'MD', 'PCA', 'Student']
    TSHCWname = ['RNs', "MDs", 'PCA', 'Student']
    HCW_per_patient = [RNs_per_ICU_patient,MDs_per_ICU_patient,PCA_per_ICU_patient,Student_per_ICU_patient,RNs_per_Inpatient, MDs_per_Inpatient,PCA_per_Inpatient,Student_per_Inpatient]
    HCW = {'RN_ICU_Typical_Sick_Standard': 0, "RN_ICU_Typical_Sick_ContactDroplet": 0, "RN_ICU_Typical_Sick_Airborne": 0, "RN_ICU_COVID_PUI": 0, 'RN_Inpatient_Typical_Sick_Standard': 0, "RN_Inpatient_Typical_Sick_ContactDroplet": 0, "RN_Inpatient_Typical_Sick_Airborne": 0, "RN_Inpatient_COVID_PUI": 0, "RN_COVID_PUI":0, "RN_Typical_Sick":0, "RN_Total":0,'MD_ICU_Typical_Sick_Standard': 0, "MD_ICU_Typical_Sick_ContactDroplet": 0, "MD_ICU_Typical_Sick_Airborne": 0, "MD_ICU_COVID_PUI": 0, 'MD_Inpatient_Typical_Sick_Standard': 0, "MD_Inpatient_Typical_Sick_ContactDroplet": 0, "MD_Inpatient_Typical_Sick_Airborne": 0, "MD_Inpatient_COVID_PUI": 0, "MD_COVID_PUI":0, "MD_Typical_Sick":0, "MD_Total":0, 'PCA_ICU_Typical_Sick_Standard': 0, "PCA_ICU_Typical_Sick_ContactDroplet": 0, "PCA_ICU_Typical_Sick_Airborne": 0, "PCA_ICU_COVID_PUI": 0, 'PCA_Inpatient_Typical_Sick_Standard': 0, "PCA_Inpatient_Typical_Sick_ContactDroplet": 0, "PCA_Inpatient_Typical_Sick_Airborne": 0, "PCA_Inpatient_COVID_PUI": 0, "PCA_COVID_PUI":0, "PCA_Typical_Sick":0, "PCA_Total":0, 'Student_ICU_Typical_Sick_Standard': 0, "Student_ICU_Typical_Sick_ContactDroplet": 0, "Student_ICU_Typical_Sick_Airborne": 0, "Student_ICU_COVID_PUI": 0, 'Student_Inpatient_Typical_Sick_Standard': 0, "Student_Inpatient_Typical_Sick_ContactDroplet": 0, "Student_Inpatient_Typical_Sick_Airborne": 0, "Student_Inpatient_COVID_PUI": 0, "Student_COVID_PUI":0, "Student_Typical_Sick":0, "Student_Total":0}

    for i in range(len(HCWname)):
        HCW[HCWname[i] + '_' +'ICU_Typical_Sick_Standard'] = staff_ICU_Typical_Sick_Standard(static[TSHCWname[i]+"_per_ICU_patient"])
        HCW[HCWname[i] + '_' +'ICU_Typical_Sick_ContactDroplet'] = staff_ICU_Typical_Sick_ContactDroplet(static[TSHCWname[i]+"_per_ICU_patient"])
        HCW[HCWname[i] + '_' +'ICU_Typical_Sick_Airborne'] = staff_ICU_Typical_Sick_Airborne()
        HCW[HCWname[i] + '_' +'ICU_COVID_PUI'] = staff_ICU_COVID_PUI(HCW_per_patient[i])
        HCW[HCWname[i] + '_' +'Inpatient_Typical_Sick_Standard'] = staff_Inpatient_Typical_Sick_Standard(static[TSHCWname[i]+"_per_Inpatient"])
        HCW[HCWname[i] + '_' +'Inpatient_Typical_Sick_ContactDroplet'] = staff_Inpatient_Typical_Sick_ContactDroplet(static[TSHCWname[i]+"_per_Inpatient"])
        HCW[HCWname[i] + '_' +'Inpatient_Typical_Sick_Airborne'] = staff_Inpatient_Typical_Sick_Airborne()
        HCW[HCWname[i] + '_' +'Inpatient_COVID_PUI'] = staff_Inpatient_COVID_PUI(HCW_per_patient[i + int(len(HCW_per_patient)/2)])
        HCW[HCWname[i] + '_' +'COVID_PUI'] = HCW[HCWname[i] + '_' +'ICU_COVID_PUI'] + HCW[HCWname[i] + '_' +'Inpatient_COVID_PUI']
        HCW[HCWname[i] + '_' +'Typical_Sick'] = HCW[HCWname[i] + '_' +'ICU_Typical_Sick_Standard'] + HCW[HCWname[i] + '_' +'ICU_Typical_Sick_ContactDroplet'] + HCW[HCWname[i] + '_' +'ICU_Typical_Sick_Airborne'] + HCW[HCWname[i] + '_' +'Inpatient_Typical_Sick_Standard'] + HCW[HCWname[i] + '_' +'Inpatient_Typical_Sick_ContactDroplet'] + HCW[HCWname[i] + '_' +'Inpatient_Typical_Sick_Airborne']
        HCW[HCWname[i] + '_' +'Total'] = HCW[HCWname[i] + '_' +'COVID_PUI'] + HCW[HCWname[i] + '_' +'Typical_Sick']
    
    Non_Clinical_COVID_PUI = round((stored['COVID_in_ICU'] + stored['PUI_in_ICU'] + stored['PUI_in_Inpatient'] + stored['COVID_inpatient'])* Non_Clinical_per_patient) * static['Shifts_per_day_non_clinical']
    Non_Clinical_Typical_Sick_Standard = round((Standard_ICU + Standard_Inpatients) * static['Non_Clinical_per_patient']) * static['Shifts_per_day_non_clinical']
    Non_Clinical_Typical_Sick_ContactDroplet = round((Contact_Droplet_ICU + Contact_Droplet_Inpatients) * static['Non_Clinical_per_patient']) * static['Shifts_per_day_non_clinical']
    Non_Clinical_Typical_Sick_Airborne = round((Airborne_ICU + Airborne_Inpatients) * 1) * static['Shifts_per_day_non_clinical']
    Non_Clinical_Typical_Sick = Non_Clinical_Typical_Sick_Standard + Non_Clinical_Typical_Sick_ContactDroplet + Non_Clinical_Typical_Sick_Airborne
    
    ED_MDs = static['ED_MDs_per_day']
    ED_RNs = static['ED_RNs_per_day']
    ED_Techs = static['ED_Techs_per_day']
    ED_Staff = ED_MDs + ED_RNs + ED_Techs
    
    Total_HCWs = HCW['RN_Total'] + HCW['MD_Total'] + HCW['PCA_Total'] + HCW['Student_Total'] + ED_Staff + static['Specialists']
    Staff_COVID_PUI = HCW['RN_COVID_PUI'] + HCW['MD_COVID_PUI'] + HCW['PCA_COVID_PUI'] + HCW['Student_COVID_PUI'] + Non_Clinical_COVID_PUI + ED_Staff + static['Specialists']
    Staff_Typical_Sick = HCW['RN_Typical_Sick'] + HCW['MD_Typical_Sick'] + HCW['PCA_Typical_Sick'] + HCW['Student_Typical_Sick'] + Non_Clinical_Typical_Sick
    
    Airborne_Staff = HCW['RN_ICU_Typical_Sick_Airborne'] + HCW['RN_Inpatient_Typical_Sick_Airborne'] + HCW['MD_ICU_Typical_Sick_Airborne'] + HCW['MD_Inpatient_Typical_Sick_Airborne'] + HCW['PCA_ICU_Typical_Sick_Airborne'] + HCW['PCA_Inpatient_Typical_Sick_Airborne'] + HCW['Student_ICU_Typical_Sick_Airborne'] + HCW['Student_Inpatient_Typical_Sick_Airborne'] + Non_Clinical_Typical_Sick_Airborne
    Contact_Droplet_Staff = HCW['RN_ICU_Typical_Sick_ContactDroplet'] + HCW['RN_Inpatient_Typical_Sick_ContactDroplet'] + HCW['MD_ICU_Typical_Sick_ContactDroplet'] + HCW['MD_Inpatient_Typical_Sick_ContactDroplet'] + HCW['PCA_ICU_Typical_Sick_ContactDroplet'] + HCW['PCA_Inpatient_Typical_Sick_ContactDroplet'] + HCW['Student_ICU_Typical_Sick_ContactDroplet'] + HCW['Student_Inpatient_Typical_Sick_ContactDroplet'] + Non_Clinical_Typical_Sick_ContactDroplet

#calculates outpatient staff
    RNs_Outpatient = round(static['Daily_outpatient_typical_sick'] * static['RNs_per_outpatient'])
    RNs_Outpatient_COVID_PUI = round(RNs_Outpatient * Cohorting)
    RNs_Outpatient_Typical_Sick = RNs_Outpatient - RNs_Outpatient_COVID_PUI
    
    MDs_Outpatient = round(static['Daily_outpatient_typical_sick'] * static['MDs_per_outpatient'])
    MDs_Outpatient_COVID_PUI = round(MDs_Outpatient * Cohorting)
    MDs_Outpatient_Typical_Sick = MDs_Outpatient - MDs_Outpatient_COVID_PUI
    
    Total_HCWs_OUT = RNs_Outpatient + MDs_Outpatient
    
#PPE calculations

    #inpatient PPE calculations
    N95s_IN = round((Staff_COVID_PUI + Airborne_Staff) / static['Shifts_per_N95'] + AGP_HCWs)
    Eye_protection_IN = round((Staff_COVID_PUI + Airborne_Staff) / static['Shifts_per_eyepro'])
    Gowns_IN = round(((stored['COVID_in_ICU'] + stored['PUI_in_ICU'] + Contact_Droplet_ICU) * static['HCW_daily_visits_ICU']) + ((stored['COVID_inpatient'] + stored['PUI_in_ICU'] + Contact_Droplet_Inpatients)* static['HCW_daily_visits_Inpatient']) + (static['Daily_patients_presenting_to_ED'] * static['Percent_ED_patient_COVID_risk'] * static['HCW_visits_per_patient_ED']) + AGP_HCWs)    
    Gloves_IN = 2 * round(((stored['COVID_in_ICU'] + stored['Typical_sick_in_ICU']) * static['HCW_daily_visits_ICU']) + ((stored['COVID_inpatient'] + static['Daily_inpatient_typical_sick']) * static['HCW_daily_visits_Inpatient']) + (static['Daily_patients_presenting_to_ED'] * static['HCW_visits_per_patient_ED']) + AGP_HCWs)
    Surg_Proc_mask_IN = round(((Total_HCWs + static['Misc_non_clinical_entering_hospital'])/static['Shifts_per_surgical_mask']) + static['Daily_patients_presenting_to_ED'] + (static['Daily_inpatient_typical_sick'] + stored['COVID_inpatient']) + (Contact_Droplet_ICU * static['HCW_daily_visits_ICU']) + (Contact_Droplet_Inpatients * static['HCW_daily_visits_Inpatient']))
    
    #outpatient PPE calculations
    N95s_OUT = round((RNs_Outpatient_COVID_PUI + MDs_Outpatient_COVID_PUI)/ static['Shifts_per_N95'])
    Eye_protection_OUT = round((RNs_Outpatient_COVID_PUI + MDs_Outpatient_COVID_PUI)/ static['Shifts_per_eyepro'])
    Gowns_OUT = round(static['Daily_outpatient_typical_sick']* Cohorting * static['HCW_visits_per_outpatient'])
    Gloves_OUT = 2 * round(static['Daily_outpatient_typical_sick'] * static['HCW_visits_per_outpatient'])
    Surg_Proc_mask_OUT = round(static['Daily_outpatient_typical_sick'] + ((RNs_Outpatient + MDs_Outpatient)/ static['Shifts_per_surgical_mask']))
                                                         
#updates the outputs
    outputs = outputs.append({'airborne_staff': Airborne_Staff,'Cohorting Coefficient': Cohorting,'New COVID Cases':daily_inputs.at[day,'New_COVID_Cases'],'COVID Patients Entering Hospital':daily_inputs.at[day,'COVID_entering_hospital'],'Total COVID in Hospital': stored['COVID_in_ICU'] + stored['COVID_inpatient'],'Total PUI in Hospital': stored['PUI_in_ICU'] + stored['PUI_in_Inpatient'],'Date': daily_inputs.at[day, 'Date'],'COVID_patients_in_ICU' : round(stored['COVID_in_ICU']) , 'typical_sick_patients_in_ICU': round(stored['Typical_sick_in_ICU']), 'typical_sick_ICU_currently_PUI':stored['PUI_in_ICU'], 'total_patients_in_ICU' : round(stored['Typical_sick_in_ICU'] + stored['COVID_in_ICU']),'COVID_patients_in_inpatient' : round(stored['COVID_inpatient']), 'typical_sick_in_inpatient': typical_sick_inpatient, 'typical_sick_inpatient_currently_PUI': stored['PUI_in_Inpatient'],'total_patients_in_inpatient' : round(inpatient_total), 'inpatients_deferred': typical_sick_inpatients_deferred, 'staff_working_with_COVID_PUI' : Staff_COVID_PUI, 'staff_working_with_typical_sick' : Staff_Typical_Sick, 'total_HCWs_working_IN' : Total_HCWs, 'total_HCWs_working_OUT' : Total_HCWs_OUT, 'outpatient_staff_COVID_PUI': (RNs_Outpatient_COVID_PUI + MDs_Outpatient_COVID_PUI),'outpatients_COVID_risk': static['Daily_outpatient_typical_sick'] * Cohorting ,'N95s_used_IN': N95s_IN, 'eye_pro_used_IN': Eye_protection_IN, 'gowns_used_IN': Gowns_IN, 'gloves_used_IN': Gloves_IN, 'surg_proc_masks_used_IN': Surg_Proc_mask_IN, 'N95s_used_OUT': N95s_OUT, 'eye_pro_used_OUT': Eye_protection_OUT, 'gowns_used_OUT': Gowns_OUT, 'gloves_used_OUT': Gloves_OUT, 'surg_proc_masks_used_OUT': Surg_Proc_mask_OUT}, ignore_index = True)
    day += 1
    
print("--- %s seconds ---" % (time.time() - start_time))                                                                 

--- 7321.311738729477 seconds ---


In [44]:
outputs.to_excel(#insert path and file name for output excel here#)